# Fluxonium with multi-path coupling(Simultaneous CNOT gate)

This notebook is similar but not as detail as `examples/fluxonium_multipath_coupling/fluxonium_mpcoupling_6q_x.ipynb`. Please check that notebook for more info.

This example is based on Nguyen, L. B. et al. Blueprint for a High-Performance Fluxonium Quantum Processor. PRX Quantum 3, 037001 (2022).
We simulate a 6 fluxonium qubit system from an underlying periodic lattice.
Idling hamiltonian of the system is
$$ H(0) = \sum_i H_{\mathrm{f},i}  + \sum_{\langle i,j \rangle } H_{ij}. $$
Hamiltonian of single fluxonium is
$$ H_{\mathrm{f},i} = 4E_{\mathrm{C},i} n_i^2 + \frac{1}{2} E_{\mathrm{L},i} (\varphi_i +\varphi_{\mathrm{ext},i})^2
    -E_{\mathrm{J},i}\cos \left( \varphi_i \right).$$
The coupling terms have the form
$$H_{ij} = J_{\mathrm{C}} n_i n_j - J_{\mathrm{L}} \varphi_i \varphi_j.$$
The couplings are chosen in a way such that the idling $ZZ$-crosstalk is almost zero.

In [1]:
import jax
import jax.numpy as jnp

from supergrad.helper import Evolve
from supergrad.utils import tensor, permute, compute_fidelity_with_1q_rotation_axis
from supergrad.utils.optimize import scipy_minimize, adam_opt
from supergrad.utils.gates import cnot
from supergrad.utils.format_conv import tree_print

from supergrad.scgraph.graph_mpc_fluxonium_5x5_periodic import CNOTGatePeriodicGraphOpt

truncated_dim = 3  # how many levels we keep after diagonalization
add_random = True  # whether to add random deviations to fluxonium parameters ec,ej,el
share_params = True  # whether we treat marked parameters as shared when computing gradients
unify_coupling = True  # set all couplings to be equal

compensation_option = 'arbit_single'  # allow arbitary single-qubit gate compensation
# This is useful for checking how many errors are fundamentally multi-qubit
# Note this option is not used here, this comes from the graph data itself now.

# This specify one order of the qubits such that the simultaneous gates are [cnot()] * 3
unitary_order = ['q02', 'q03', 'q12', 'q13', 'q22', 'q23']

# instance the quantum processor graph, and choose a subset for time evolution
gate_graph = CNOTGatePeriodicGraphOpt(seed=1)

# Apply truncated_dim to fluxonium
gate_graph.set_all_node_attr(truncated_dim=truncated_dim)

# Apply random deviations to fluxonium parameters
if add_random:
    gate_graph.add_lcj_params_variance_to_graph()

# Set share_params and unify_coupling
qubit_subset = gate_graph.subscgraph(unitary_order)
qubit_subset.share_params = share_params
qubit_subset.unify_coupling = unify_coupling



opt = 'adam'

evo = Evolve(qubit_subset)

# Compute the order of qubits in qubit_subset related to unitary_order
qubit_order = [unitary_order.index(key) for key in qubit_subset.sorted_nodes]
# Compute the corresponding target unitary based on the above order
target_unitary = permute(tensor(*([cnot()] * 3)), [2] * len(qubit_subset.nodes), qubit_order)


Let us print the parameters stored in the evo instance. These parameters are parsed from gate_graph. For how we construct this graph, please check `examples/fluxonium_multipath_coupling/graph_5x5_periodic.py` and the instructions inside.

In [2]:
# This is the same as  evo.all_params
params = qubit_subset.convert_graph_to_parameters()
tree_print(params)


{ 'edges': { 'q02:q03': { 'capacitive_coupling': {'strength': 0.07225663},
                          'inductive_coupling': {'strength': -0.01256637}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': [ -0.01528741,
                                                       0.04418319,
                                                       -0.374539],
                                        'pre_comp': [ 0.05671005,
                                                      0.02682845,
                                                      0.0395099]},
                      'ec': 6.3532393823,
                      'ej': 25.1103087675,
                      'el': 6.344787656,
                      'pulse': { 'p1': { 'amp': 0.18128846,
                                         'omega_d': 2.58934559,
                                         'phase': -0.24290228,
                                         't_plateau': 69.93608145,
                                         't_ramp': 29.92806488}}},


For optimization, let us use an object function based on the average gate fidelity with leakage. The formula is from Physical Review A 87, 022309 (2013).
$$
	C = 1 - \frac{1}{D(D + 1)}[Tr(PU^{\dagger}_{\text{sim}}P U_{\text{sim}}P)+|Tr(PU^{\dagger}_{\text{sim}} P U_{\text{target}})|^2],   
$$
where $P$ is the projector into the computational subspace $S$, and $D=2^n$ is the dimension of $S$.

In [3]:
def infidelity(params, target_unitary):
    # Compute the time evolution unitary in the eigenbasis.
    sim_u = evo.eigen_basis(params)
    # calculate fidelity
    fidelity_vz, _ = compute_fidelity_with_1q_rotation_axis(target_unitary,
                                                            sim_u,
                                                            opt_method=None)

    return jnp.log10(1 - fidelity_vz)


In [4]:
vg = jax.value_and_grad(infidelity)(params, target_unitary)
print('The infidelity of simultaneous CNOT gates is ', 10**vg[0])


The infidelity of simultaneous CNOT gates is  0.0006533230929488555


In [5]:
print('The gradient of log10(infidelity):')
tree_print(vg[1])


The gradient of log10(infidelity):
{ 'edges': { 'q02:q03': { 'capacitive_coupling': { 'strength': -0.30630691703423857},
                          'inductive_coupling': { 'strength': -158.68196537187967}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': [ -0.0430580597485069,
                                                       -0.10818519720943698,
                                                       -0.00047244962298532544],
                                        'pre_comp': [ 0.061237309286806256,
                                                      0.006098037087461705,
                                                      0.00027790081945203095]},
                      'ec': 0.43265623330076153,
                      'ej': 0.1302493167203438,
                      'el': -0.15590224582613565,
                      'pulse': { 'p1': { 'amp': -2.205376779033003,
                                         'omega_d': 5.542185169560069,
                                         

Show the optimization procedure of control parameters

In [6]:
adam_opt(infidelity,
            params,
            args=(target_unitary,),
            options={
                'adam_lr': 1e-6,
                'steps': 3
            })


{'adam_lr': 1e-06, 'adam_lr_decay_rate': 1000, 'steps': 3, 'adam_b1': 0.9, 'adam_b2': 0.999}
step: 0
parameters:
{ 'edges': { 'q02:q03': { 'capacitive_coupling': {'strength': 0.07225663},
                          'inductive_coupling': {'strength': -0.01256637}}},
  'nodes': { 'q02': { 'compensation': { 'post_comp': [ -0.01528741,
                                                       0.04418319,
                                                       -0.374539],
                                        'pre_comp': [ 0.05671005,
                                                      0.02682845,
                                                      0.0395099]},
                      'ec': 6.3532393823,
                      'ej': 25.1103087675,
                      'el': 6.344787656,
                      'pulse': { 'p1': { 'amp': 0.18128846,
                                         'omega_d': 2.58934559,
                                         'phase': -0.24290228,
                     

{'edges': {('q02',
   'q03'): {'capacitive_coupling': {'strength': Array(0.07225773, dtype=float64)}, 'inductive_coupling': {'strength': Array(-0.01256337, dtype=float64)}}},
 'nodes': {'q02': {'compensation': {'post_comp': Array([-0.01528441,  0.04418611, -0.374536  ], dtype=float64),
    'pre_comp': Array([0.05670705, 0.02682877, 0.0395069 ], dtype=float64)},
   'ec': Array(6.3532378, dtype=float64),
   'ej': Array(25.11030808, dtype=float64),
   'el': Array(6.34478826, dtype=float64),
   'pulse': {'p1': {'amp': Array(0.1812913, dtype=float64),
     'omega_d': Array(2.58934339, dtype=float64),
     'phase': Array(-0.24290522, dtype=float64),
     't_plateau': Array(69.93608185, dtype=float64),
     't_ramp': Array(29.92806514, dtype=float64)}}},
  'q03': {'compensation': {'post_comp': Array([-1.0062949 ,  0.16410731, -0.00797813], dtype=float64),
    'pre_comp': Array([-1.46070293,  0.66029055,  0.0660364 ], dtype=float64)},
   'ec': Array(6.25506345, dtype=float64),
   'ej': Array(2